In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re, os, pickle

In [2]:
dataset_folder = "C:\\Users\\Marios\\Documents\\GitHub\\Thomas Prosser project\\datasets\\"
output_folder = "C:\\Users\\Marios\\Documents\\GitHub\\Thomas Prosser project\\output\\"

In [3]:
partyfacts_core_parties = pd.read_csv(dataset_folder+"PartyFacts"+os.sep+ "partyfacts-core-parties.csv")
partyfacts_external_parties = pd.read_csv(dataset_folder+"PartyFacts"+os.sep+ "partyfacts-external-parties.csv")
partyfacts_countries = pd.read_csv(dataset_folder+"PartyFacts"+os.sep+ "partyfacts-countries.csv")
partyfacts_datasets = pd.read_csv(dataset_folder+"PartyFacts"+os.sep+ "partyfacts-datasets.csv")

pf_country_to_code = partyfacts_countries[["country","name"]].set_index("name")["country"].apply(lambda x: x.lower())

In [160]:
# df = pd.read_stata( dataset_folder + "DPI" + os.sep + "DPI2017_basefile.dta" )
dpi = pd.read_stata( dataset_folder + "DPI" + os.sep + "DPI2017.dta" )

# df2 identical to df
# except it has a bunch more columns
# and on three columns there a few rows in which -999 is replace with 0

reader = pd.io.stata.StataReader( dataset_folder + "DPI" + os.sep +"DPI2017.dta" )
header = reader.variable_labels()
labels = reader.value_labels()

In [ ]:
## It's really a COUNTRY x YEAR dataset about executives

# But it lists up to 3+Other "governing parties"
# And it categorises the head the executive in party terms (unless the party leader really deviates)

In [150]:
dpi_to_pf_ids = partyfacts_external_parties[partyfacts_external_parties["dataset_key"]=="dpi"][["dataset_party_id","partyfacts_id"]].set_index("dataset_party_id")

# [x for x in dpi.columns if "party" in x]

# dpi_to_pf_ids
dpi_to_pf_ids.index = [int(x) for x in dpi_to_pf_ids.index]
dpi_to_pf_ids.index.min(),dpi_to_pf_ids.index.max(),

(1, 2331)

In [168]:
dpi[dpi["gov1me"]=="PP"]

,countryname,ifs,year,system,yrsoffc,finittrm,yrcurnt,multpl,military,defmin,...,checks,stabs_strict,stabs,stabns_strict,stabns,tenlong_strict,tenlong,tenshort_strict,tenshort,polariz
2217,Spain,ESP,1997-01-01 00:00:00,Parliamentary,1.0,1.0,3.0,1.0,0.0,0.0,...,4,0.5,0.5,0.666667,0.666667,1,1,1,1,2
2218,Spain,ESP,1998-01-01 00:00:00,Parliamentary,2.0,1.0,2.0,1.0,0.0,0.0,...,4,0,0,0,0,2,2,2,2,2
2219,Spain,ESP,1999-01-01 00:00:00,Parliamentary,3.0,1.0,1.0,1.0,0.0,0.0,...,4,0,0,0,0,3,3,3,3,2
2220,Spain,ESP,2000-01-01 00:00:00,Parliamentary,4.0,1.0,0.0,1.0,0.0,0.0,...,4,0,0,0,0,4,4,4,4,2
2221,Spain,ESP,2001-01-01 00:00:00,Parliamentary,5.0,1.0,3.0,1.0,0.0,0.0,...,3,0.6,0.6,0.5,0.5,5,5,5,5,0
2222,Spain,ESP,2002-01-01 00:00:00,Parliamentary,6.0,1.0,2.0,1.0,0.0,0.0,...,3,0,0,0,0,6,6,6,6,0
2223,Spain,ESP,2003-01-01 00:00:00,Parliamentary,7.0,1.0,1.0,1.0,0.0,0.0,...,3,0,0,0,0,7,7,7,7,0
2224,Spain,ESP,2004-01-01 00:00:00,Parliamentary,8.0,1.0,0.0,1.0,0.0,0.0,...,3,0,0,0,0,8,8,8,8,0
2232,Spain,ESP,2012-01-01 00:00:00,Parliamentary,1.0,1.0,3.0,1.0,0.0,0.0,...,3,0.75,0.75,0.666667,0.666667,1,1,1,1,0
2233,Spain,ESP,2013-01-01 00:00:00,Parliamentary,2.0,1.0,2.0,1.0,0.0,0.0,...,3,0,0,0,0,2,2,2,2,0


In [170]:
# dpi.describe().loc["max"].sort_values()

# dpi.groupby("gov1me")["countryname"].apply(lambda x: len(x.unique())).sort_values().tail(50)

list(dpi.columns)

['countryname',
 'ifs',
 'year',
 'system',
 'yrsoffc',
 'finittrm',
 'yrcurnt',
 'multpl',
 'military',
 'defmin',
 'percent1',
 'percentl',
 'prtyin',
 'execme',
 'execrlc',
 'execnat',
 'execrurl',
 'execreg',
 'execrel',
 'execage',
 'allhouse',
 'nonchief',
 'totalseats',
 'gov1me',
 'gov1seat',
 'gov1vote',
 'gov1rlc',
 'gov1nat',
 'gov1rurl',
 'gov1reg',
 'gov1rel',
 'gov1age',
 'gov2me',
 'gov2seat',
 'gov2vote',
 'gov2rlc',
 'gov2nat',
 'gov2rurl',
 'gov2reg',
 'gov2rel',
 'gov2age',
 'gov3me',
 'gov3seat',
 'gov3vote',
 'gov3rlc',
 'gov3nat',
 'gov3rurl',
 'gov3reg',
 'gov3rel',
 'gov3age',
 'govoth',
 'govothst',
 'govothvt',
 'opp1me',
 'opp1seat',
 'opp1vote',
 'opp1rlc',
 'opp1nat',
 'opp1rurl',
 'opp1reg',
 'opp1rel',
 'opp1age',
 'opp2me',
 'opp2seat',
 'opp2vote',
 'opp3me',
 'opp3seat',
 'opp3vote',
 'oppoth',
 'oppothst',
 'oppothvt',
 'ulprty',
 'numul',
 'ulvote',
 'oppmajh',
 'oppmajs',
 'dateleg',
 'dateexec',
 'legelec',
 'exelec',
 'liec',
 'eiec',
 'mdmh',
 'm

In [159]:
dpi.select_dtypes('category')
# year -> datetime

# {" ":np.nan, "-999":np.nan, -999:np.nan}

,system,execrlc,execrel,gov1rlc,gov1rel,gov2rlc,gov2rel,gov3rlc,gov3rel,opp1rlc,...,checks,stabs_strict,stabs,stabns_strict,stabns,tenlong_strict,tenlong,tenshort_strict,tenshort,polariz
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-999,0,-999,-999,-999,-999,-999,-999,-999,-999,...,NA,NaN,NaN,NaN,NaN,NA,NA,NA,NA,NA
3,Presidential,0,0,-999,-999,-999,-999,-999,-999,-999,...,1,NA,NA,NA,NA,1,1,1,1,0
4,Presidential,Right,0,0,-999,-999,-999,-999,-999,Left,...,3,0,0,0,0,2,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7658,Presidential,0,0,0,0,-999,-999,-999,-999,Left,...,4,0,0,0,0,26,26,5,5,NaN
7659,Presidential,0,0,0,0,-999,-999,-999,-999,Left,...,4,0,0,0,0,27,27,1,1,0
7660,Presidential,0,0,0,0,-999,-999,-999,-999,Left,...,4,0,0,0,0,28,28,2,2,0
7661,Presidential,0,0,0,0,-999,-999,-999,-999,Left,...,4,0,0,0,0,29,29,3,3,0


In [161]:
header

{'countryname': 'Country Name',
 'ifs': 'Country Code',
 'year': 'Year',
 'system': 'Political System',
 'yrsoffc': 'Chief Executive Years in Office',
 'finittrm': 'Finite Term in Office',
 'yrcurnt': 'Years Left in Current Term',
 'multpl': 'Can Chief Executive Serve Multiple Terms',
 'military': 'Is Chief Executive a Military Officer?',
 'defmin': 'Is Defense Minister a Military Officer?',
 'percent1': 'President Percentage of Votes, first round',
 'percentl': 'President Percentage of Votes, last round',
 'prtyin': 'Party of Chief Executive Length of Time in Office',
 'execme': 'Name of Executive Party',
 'execrlc': 'Chief Executive Party Orientation',
 'execnat': 'Chief Executive Party: Nationalist',
 'execrurl': 'Chief Executive Party: Rural',
 'execreg': 'Chief Executive Party: Regional',
 'execrel': 'Chief Executive Party: Religious',
 'execage': 'Age of Chief Executive Party',
 'allhouse': 'Does Party of Executive Control All Houses?',
 'nonchief': 'Party affiliation of Non-Chie

In [162]:
labels

{'candidate': {1: 'National', 2: 'Sub-National', 3: 'Primary'},
 'presparl': {0: 'Presidential',
  1: 'Assembly-Elected President',
  2: 'Parliamentary'},
 'sys': {0: 'PR', 1: 'Plurality'},
 'fed': {0: 'No local elections',
  1: 'Legislature locally elected',
  2: 'Legislature and executive locally elected'},
 'rel': {1: 'Christian',
  2: 'Catholic',
  3: 'Islamic',
  4: 'Hindu',
  5: 'Buddhist',
  6: 'Jewish'},
 'rlc': {1: 'Right', 2: 'Center', 3: 'Left'},
 'NA': {-999: 'NA'}}

In [ ]:
### CLEA - Constituency-Level Elections Archive
# datasets:
# http://www.electiondataarchive.org/data-and-documentation.php

# whole bunch of election level datasets

# of immediate relevance is party-level dataset on "Nationalization"
# split into upper and lower chamber measures
# split into constituency/national/party-levels

In [4]:
clea_lc_party = pd.read_stata( dataset_folder + "CLEA" + os.sep + "Lower Chamber Nationalization" + os.sep + "clea_lc_enp_20190617_party.level.dta" )

clea_uc_party = pd.read_stata( dataset_folder + "CLEA" + os.sep + "Upper Chamber Nationalization" + os.sep + "clea_uc_enp_20190617_party.level.dta" )



In [5]:
clea_lc_party

,id,ctr_n,ctr,yr,mn,cst_tot,pty_n,pty,PNS,PNS_s,PNS_w,PNS_sw
0,-999.0,US,840.0,1788.0,Missing data,293.0,independent,6001,-990.0,1.0,-990.0,-990.0
1,-999.0,US,840.0,1788.0,Missing data,293.0,independent,6002,-990.0,1.0,-990.0,-990.0
2,-999.0,US,840.0,1788.0,Missing data,293.0,independent,6003,-990.0,1.0,-990.0,-990.0
3,-999.0,US,840.0,1788.0,Missing data,293.0,independent,6004,-990.0,1.0,-990.0,-990.0
4,-999.0,US,840.0,1788.0,Missing data,293.0,independent,6005,-990.0,1.0,-990.0,-990.0
...,...,...,...,...,...,...,...,...,...,...,...,...
143050,1834.0,United Arab Emirates,784.0,2011.0,September,7.0,Independent,6037,-990.0,-990.0,-990.0,-990.0
143051,1834.0,United Arab Emirates,784.0,2011.0,September,7.0,Independent,6038,-990.0,-990.0,-990.0,-990.0
143052,1834.0,United Arab Emirates,784.0,2011.0,September,7.0,Independent,6039,-990.0,-990.0,-990.0,-990.0
143053,1834.0,United Arab Emirates,784.0,2011.0,September,7.0,Independent,6040,-990.0,-990.0,-990.0,-990.0


In [6]:
clea_uc_party

,id,ctr_n,ctr,yr,mn,cst_tot,pty_n,pty,PNS,PNS_s,PNS_w,PNS_sw
0,1.0,Argentina,32.0,2001.0,October,24.0,ACCION CHUBUTENSE,345,-990.000000,-990.000000,0.010395,0.002446
1,1.0,Argentina,32.0,2001.0,October,24.0,ACCION CIVICA PROVINCIAL,3,-990.000000,-990.000000,0.012941,0.003265
2,1.0,Argentina,32.0,2001.0,October,24.0,ACCION NATIVA,105,-990.000000,-990.000000,0.012941,0.003265
3,1.0,Argentina,32.0,2001.0,October,24.0,ACCION PARA EL CAMBIO,4,-990.000000,-990.000000,0.090026,0.041987
4,1.0,Argentina,32.0,2001.0,October,24.0,ACCION POPULAR,106,-990.000000,-990.000000,0.090026,0.041987
...,...,...,...,...,...,...,...,...,...,...,...,...
5408,101.0,Zimbabwe,716.0,2013.0,July,10.0,ZANU-Ndonga,21,-990.000000,-990.000000,0.263965,0.245746
5409,101.0,Zimbabwe,716.0,2013.0,July,10.0,Zimbabwe African National Union - Patriotic Front,22,0.820625,0.917727,0.854098,0.846896
5410,101.0,Zimbabwe,716.0,2013.0,July,10.0,Zimbabwe African People's Union,23,-990.000000,-990.000000,0.215464,0.198417
5411,101.0,Zimbabwe,716.0,2013.0,July,10.0,Zimbabwe Development Party,25,-990.000000,-990.000000,0.162185,0.147093


In [8]:
clea_to_pf_ids = partyfacts_external_parties[partyfacts_external_parties["dataset_key"]=="clea"][["dataset_party_id","partyfacts_id"]].set_index("dataset_party_id")
clea_to_pf_ids.index = [int(x) for x in clea_to_pf_ids.index]

In [9]:
# clea_to_pf_ids.index


In [10]:
clea_lc_party.describe(include='all')

,id,ctr_n,ctr,yr,mn,cst_tot,pty_n,pty,PNS,PNS_s,PNS_w,PNS_sw
count,143055.000000,143055,143055.000000,143055.000000,143055,143055.00000,143055,143055.0,143055.000000,143055.000000,143055.000000,143055.000000
unique,NaN,168,NaN,NaN,13,NaN,16272,10033.0,NaN,NaN,NaN,NaN
top,NaN,India,NaN,NaN,Missing data,NaN,independent,6001.0,NaN,NaN,NaN,NaN
freq,NaN,42602,NaN,NaN,28726,NaN,70730,761.0,NaN,NaN,NaN,NaN
mean,1146.705444,NaN,443.059601,1983.362915,NaN,286.03125,NaN,NaN,-945.714600,-945.469421,-809.169373,-809.393982
std,678.232910,NaN,247.290054,46.528362,NaN,206.15300,NaN,NaN,204.721344,205.283386,382.594666,382.424133
min,-999.000000,NaN,4.000000,1788.000000,NaN,1.00000,NaN,NaN,-990.000000,-990.000000,-990.000000,-990.000000
25%,923.000000,NaN,356.000000,1982.000000,NaN,87.00000,NaN,NaN,-990.000000,-990.000000,-990.000000,-990.000000
50%,1423.000000,NaN,364.000000,1998.000000,NaN,253.00000,NaN,NaN,-990.000000,-990.000000,-990.000000,-990.000000
75%,1585.000000,NaN,608.000000,2009.000000,NaN,534.00000,NaN,NaN,-990.000000,-990.000000,-990.000000,-990.000000


In [64]:
clea_lc_party.shape

(143055, 12)

In [197]:
# clea_lc_party["pty"].astype(int)
#.apply(lambda x: int(x) not in clea_to_pf_ids).sum()

In [12]:
# clea_lc_party.isnull().sum()

# (clea_lc_party["pty"]=='Suspended').sum()
clea_to_pf_ids.index.min(),clea_to_pf_ids.index.max()

(4000001, 1003000003)

In [13]:
clea_lc_party["party_id"] = clea_lc_party["pty"].replace({'Suspended':np.nan}).astype(float)
mask = clea_lc_party["party_id"].notnull()

clea_lc_party["ctr_pty"] = clea_lc_party["ctr"][mask].astype(int).astype(str)+"00"+clea_lc_party["party_id"][mask].apply(lambda x: format(int(x), '04d'))
clea_lc_party["ctr_pty"].astype(float).min(),clea_to_pf_ids.index.min()

(4000001.0, 4000001)

In [18]:
clea_to_pf_ids

,partyfacts_id
4000001,5876
8000051,2570
8000001,594
8000022,190
8000008,840
...,...
894000032,4125
894000034,4124
716000007,3558
716000009,3559


(4000001.0, 4000001)

In [14]:
clea_lc_party["ctr_pty"][mask].apply(lambda x: int(x) in clea_to_pf_ids.index).sum()# in clea_to_pf_ids.index) #  in clea_to_pf_ids.index



# clea_to_pf_ids = 

9928

In [19]:
clea_lc_party["partyfacts_id"] = clea_lc_party["ctr_pty"][mask].apply(lambda x: clea_to_pf_ids.loc[int(x),"partyfacts_id"] if int(x) in clea_to_pf_ids.index else np.nan)

In [20]:
clea_lc_party[clea_lc_party["partyfacts_id"].notnull()]


,id,ctr_n,ctr,yr,mn,cst_tot,pty_n,pty,PNS,PNS_s,PNS_w,PNS_sw,party_id,ctr_pty,partyfacts_id
37,-999.0,US,840.0,1789.0,Missing data,293.0,federalist,218,-990.000000,-990.000000,0.039662,4.199593e-04,218.0,840000218,4517.0
142,-999.0,US,840.0,1790.0,Missing data,293.0,democratic republican,186,-990.000000,-990.000000,0.024570,5.286311e-13,186.0,840000186,4538.0
143,-999.0,US,840.0,1790.0,Missing data,293.0,federalist,218,-990.000000,-990.000000,0.024570,5.286314e-13,218.0,840000218,4517.0
205,-999.0,US,840.0,1791.0,Missing data,293.0,democratic republican,186,-990.000000,-990.000000,0.064202,2.215181e-04,186.0,840000186,4538.0
206,-999.0,US,840.0,1791.0,Missing data,293.0,federalist,218,-990.000000,-990.000000,0.064202,2.215181e-04,218.0,840000218,4517.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142984,1830.0,Trinidad and Tobago,780.0,1971.0,May,36.0,People's National Movement,14,0.924027,0.976567,0.930350,9.506890e-01,14.0,780000014,4283.0
142990,1831.0,Trinidad and Tobago,780.0,1976.0,September,36.0,People's National Movement,14,0.791108,0.936704,0.784771,8.550876e-01,14.0,780000014,4283.0
143003,1832.0,Trinidad and Tobago,780.0,1981.0,November,36.0,People's National Movement,14,0.834184,0.950665,0.832441,8.884526e-01,14.0,780000014,4283.0
143010,1833.0,Trinidad and Tobago,780.0,1986.0,December,36.0,National Alliance for Reconstruction,7,0.885410,0.966608,0.887457,9.256927e-01,7.0,780000007,4284.0


In [208]:
clea_to_pf_ids
clea_lc_party

# ctr_pty
# 4000001
# 4|00|0001

,id,ctr_n,ctr,yr,mn,cst_tot,pty_n,pty,PNS,PNS_s,PNS_w,PNS_sw
0,-999.0,US,840.0,1788.0,Missing data,293.0,independent,6001,-990.0,1.0,-990.0,-990.0
1,-999.0,US,840.0,1788.0,Missing data,293.0,independent,6002,-990.0,1.0,-990.0,-990.0
2,-999.0,US,840.0,1788.0,Missing data,293.0,independent,6003,-990.0,1.0,-990.0,-990.0
3,-999.0,US,840.0,1788.0,Missing data,293.0,independent,6004,-990.0,1.0,-990.0,-990.0
4,-999.0,US,840.0,1788.0,Missing data,293.0,independent,6005,-990.0,1.0,-990.0,-990.0
...,...,...,...,...,...,...,...,...,...,...,...,...
143050,1834.0,United Arab Emirates,784.0,2011.0,September,7.0,Independent,6037,-990.0,-990.0,-990.0,-990.0
143051,1834.0,United Arab Emirates,784.0,2011.0,September,7.0,Independent,6038,-990.0,-990.0,-990.0,-990.0
143052,1834.0,United Arab Emirates,784.0,2011.0,September,7.0,Independent,6039,-990.0,-990.0,-990.0,-990.0
143053,1834.0,United Arab Emirates,784.0,2011.0,September,7.0,Independent,6040,-990.0,-990.0,-990.0,-990.0


In [204]:
partyfacts_external_parties[partyfacts_external_parties["dataset_key"]=="clea"][["dataset_party_id","partyfacts_id"]]

,dataset_party_id,partyfacts_id
6951,4000001,5876
6952,8000051,2570
6953,8000001,594
6954,8000022,190
6955,8000008,840
...,...,...
9203,894000032,4125
9204,894000034,4124
9205,716000007,3558
9206,716000009,3559


In [ ]:
## POLCON

# COUNTRY x YEAR (but lists parties)

In [24]:
encoding = "ISO-8859-1"
polcon = pd.read_csv( dataset_folder + "POLCON" + os.sep +"POLCON_2017.csv" , encoding=encoding)

# I don't see any party_ids in this - or any previous - datset

# polcon_ids.csv

polcon_ids = pd.read_csv( dataset_folder + "POLCON" + os.sep + "polcon_ids.csv" )

C:\ProgramData\Anaconda3\envs\BES_analysis\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (27,31,63,83,85,87,89,91,93) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [134]:
# polcon.columns[[27,31,63,83,85,87,89,91,93]]
# polcon_ids

In [77]:
polcon2012 = pd.read_stata( dataset_folder + "POLCON" + os.sep +"polcon2012.dta" )
reader = pd.io.stata.StataReader( dataset_folder + "POLCON" + os.sep +"polcon2012.dta" )
header = reader.variable_labels()
labels = reader.value_labels()

# There isn't really a codebook to go with this and explain what the variables are

# we have a country x year level dataset
# country-year ids
# 'cyear', 'ccode', 'cnts_country', 'polity_country', 'icrg_country', 'ctrynm', 'year',

# polcon index variables
# 'polconiii', 'polconv', 'polconvj',

# binary flags???
# 'l1', 'l2', 'j',  'f', 'aligne_l1', 'aligne_l2', 'alignl1_l2', 'aligne_j',

# some sort of fraction
# 'legfralower', 'legfraupper', 'judfra',

# 
# 'laworderfromicrg', 'xconstfrompolity', 'polityfrompolity',

#'executivename','executiveparty','executiveelectedby','executivevoteshare','primeministername','primeministerparty','executivepartydom',


# two sets of seat lists
# partycount(lower|upper), ts(l|u), X=1-15 (party(X)name(lower|upper),pX(l|u))



In [93]:
polcon[polcon.columns[18:30]].describe(include='all')

,legfralower,legfraupper,judfra,laworderfromicrg,xconstfrompolity,polityfrompolity,executivename,executiveparty,executiveelectedby,executivevoteshare,primeministername,primeministerparty
count,10154.000000,2726.000000,1297.000000,4237.000000,16381.000000,16166.000000,17461,17076,13512,2987,17934,16593
unique,NaN,NaN,NaN,NaN,NaN,NaN,2005,496,18,122,2103,541
top,NaN,NaN,NaN,NaN,NaN,NaN,?,?,Monarchy,100,na,na
freq,NaN,NaN,NaN,NaN,NaN,NaN,2713,3816,3691,301,9572,6824
mean,0.474983,0.552929,0.302398,3.564635,0.123741,-0.384511,NaN,NaN,NaN,NaN,NaN,NaN
std,0.285792,0.241820,0.282425,1.558814,17.326464,7.041057,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,0.000000,0.000000,0.000000,-88.000000,-10.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.273722,0.494350,0.000000,2.500000,1.000000,-7.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.547820,0.588095,0.320000,4.000000,3.000000,-3.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,0.701784,0.720202,0.530000,5.000000,7.000000,7.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
polcon.columns

Index(['cyear', 'ccode', 'cnts_country', 'polity_country', 'icrg_country',
       'ctrynm', 'year', 'polconiii', 'polconv', 'polconvj', 'l1', 'l2', 'j',
       'f', 'aligne_l1', 'aligne_l2', 'alignl1_l2', 'aligne_j', 'legfralower',
       'legfraupper', 'judfra', 'laworderfromicrg', 'xconstfrompolity',
       'polityfrompolity', 'executivename', 'executiveparty',
       'executiveelectedby', 'executivevoteshare', 'primeministername',
       'primeministerparty', 'executivepartydom', 'partycountlower', 'tsl',
       'party1namelower', 'p1l', 'party2namelower', 'p2l', 'party3namelower',
       'p3l', 'party4namelower', 'p4l', 'party5namelower', 'p5l',
       'party6namelower', 'p6l', 'party7namelower', 'p7l', 'party8namelower',
       'p8l', 'party9namelower', 'p9l', 'party10namelower', 'p10l',
       'party11namelower', 'p11l', 'party12namelower', 'p12l',
       'party13namelower', 'p13l', 'party14namelower', 'p14l',
       'party15namelower', 'p15l', 'partycountupper', 'tsu', 'party1na

In [75]:
polcon2012.columns

Index(['cyear', 'ccode', 'cnts_country', 'polity_country', 'icrg_country',
       'ctrynm', 'year', 'polconiii', 'polconv', 'polconvj', 'l1', 'l2', 'j',
       'f', 'aligne_l1', 'aligne_l2', 'alignl1_l2', 'aligne_j', 'legfralower',
       'legfraupper', 'judfra', 'laworderfromicrg', 'xconstfrompolity',
       'polityfrompolity', 'executivename', 'executiveparty',
       'executiveelectedby', 'executivevoteshare', 'primeministername',
       'primeministerparty', 'executivepartydom', 'partycountlower', 'tsl',
       'party1namelower', 'p1l', 'party2namelower', 'p2l', 'party3namelower',
       'p3l', 'party4namelower', 'p4l', 'party5namelower', 'p5l',
       'party6namelower', 'p6l', 'party7namelower', 'p7l', 'party8namelower',
       'p8l', 'party9namelower', 'p9l', 'party10namelower', 'p10l',
       'party11namelower', 'p11l', 'party12namelower', 'p12l',
       'party13namelower', 'p13l', 'party14namelower', 'p14l',
       'party15namelower', 'p15l', 'partycountupper', 'tsu', 'party1na

In [128]:
polcon[(polcon[[x for x in polcon.columns if "party" in x]]["party3namelower"]=="Andorra for Change")]

,cyear,ccode,cnts_country,polity_country,icrg_country,ctrynm,year,polconiii,polconv,polconvj,...,party11nameupper,p11u,party12nameupper,p12u,party13nameupper,p13u,party14nameupper,p14u,party15nameupper,p15u
6136,2322010,232,ANDORRA,NaN,NaN,ADO,2010.0,0.4039,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
6137,2322011,232,ANDORRA,NaN,NaN,ADO,2011.0,0.4039,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0


In [21]:
polcon_to_pf_ids = partyfacts_external_parties[partyfacts_external_parties["dataset_key"]=="polcon"][["dataset_party_id","partyfacts_id"]].set_index("dataset_party_id")
polcon_to_pf_ids.index = [int(x) for x in polcon_to_pf_ids.index]

In [98]:
polcon_to_pf_ids

,partyfacts_id
dataset_party_id,
1033,5183
1046,5184
1055,190
1069,4711
1070,819
...,...
8525,3305
8527,5344
8529,5346


In [78]:
header

{'cyear': '',
 'ccode': '',
 'cnts_country': '',
 'polity_country': '',
 'icrg_country': 'ICRG_country',
 'ctrynm': 'CTRYNM',
 'year': 'Year',
 'polconiii': 'POLCONIII',
 'polconv': 'POLCONV',
 'polconvj': 'POLCONVJ',
 'l1': 'L1',
 'l2': 'L2',
 'j': 'J',
 'f': 'F',
 'aligne_l1': 'Align E_L1',
 'aligne_l2': 'Align E_L2',
 'alignl1_l2': 'Align L1_L2',
 'aligne_j': 'Align E_J',
 'legfralower': 'LEGFRA lower',
 'legfraupper': 'LEGFRA upper',
 'judfra': 'JUDFRA',
 'laworderfromicrg': 'Law & Order (from ICRG)',
 'xconstfrompolity': 'XCONST (from Polity)',
 'polityfrompolity': 'POLITY (from Polity)',
 'executivename': 'Executive name',
 'executiveparty': 'Executive party',
 'executiveelectedby': 'Executive Elected By',
 'executivevoteshare': 'Executive Vote Share',
 'primeministername': 'Prime Minister Name',
 'primeministerparty': 'Prime Minister Party',
 'executivepartydom': 'Executive Party (dom)',
 'partycountlower': 'Party count lower',
 'tsl': '',
 'party1namelower': 'Party1 name lower'

In [100]:
polcon[polcon.columns[30:40]]

,executivepartydom,partycountlower,tsl,party1namelower,p1l,party2namelower,p2l,party3namelower,p3l,party4namelower
0,Federalist,2,106.0,Federalist,64,Democrat-Republicans,42,NaN,0,NaN
1,Democratic-Republican,2,105.0,Democrat-Republicans,69,Federalist,36,NaN,0,NaN
2,Democratic-Republican,2,105.0,Democrat-Republicans,69,Federalist,36,NaN,0,NaN
3,Democratic-Republican,2,141.0,Democrat-Republicans,109,Federalist,32,NaN,0,NaN
4,Democratic-Republican,2,141.0,Democrat-Republicans,109,Federalist,32,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...
17943,Human Rights Protection Party,2,49.0,Human Rights Protection Party,36,Tautua Samoa Party,13,NaN,0,NaN
17944,Human Rights Protection Party,2,49.0,Human Rights Protection Party,36,Tautua Samoa Party,13,NaN,0,NaN
17945,Human Rights Protection Party,2,49.0,Human Rights Protection Party,36,Tautua Samoa Party,13,NaN,0,NaN
17946,Human Rights Protection Party,2,49.0,Human Rights Protection Party,36,Tautua Samoa Party,13,NaN,0,NaN


In [106]:
polcon_to_pf_ids.index.max()

1002-9028

'9028'

In [112]:
polcon_to_pf_ids[polcon_to_pf_ids["partyfacts_id"]=="8190"]

C:\ProgramData\Anaconda3\envs\BES_analysis\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,partyfacts_id
dataset_party_id,


In [119]:
polcon_to_pf_ids[(polcon_to_pf_ids["partyfacts_id"]>8000)]

,partyfacts_id
dataset_party_id,
2604,8417
3029,8132
4936,8490
5544,8013
6635,8268
7340,8186
8112,8366


In [30]:
polcon

,cyear,ccode,cnts_country,polity_country,icrg_country,ctrynm,year,polconiii,polconv,polconvj,...,party11nameupper,p11u,party12nameupper,p12u,party13nameupper,p13u,party14nameupper,p14u,party15nameupper,p15u
0,21800,2,US,United States,NaN,USA,1800.0,0.392401,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
1,21801,2,US,United States,NaN,USA,1801.0,0.385204,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
2,21802,2,US,United States,NaN,USA,1802.0,0.385204,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
3,21803,2,US,United States,NaN,USA,1803.0,0.301785,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
4,21804,2,US,United States,NaN,USA,1804.0,0.301785,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17943,9902012,990,SAMOA,NaN,NaN,WSM,2012.0,0.265319,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
17944,9902013,990,SAMOA,NaN,NaN,WSM,2013.0,0.265319,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
17945,9902014,990,SAMOA,NaN,NaN,WSM,2014.0,0.265319,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
17946,9902015,990,SAMOA,NaN,NaN,WSM,2015.0,0.265319,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0


In [31]:
polcon_to_pf_ids.loc["1002"]

KeyError: '1002'

In [124]:
partyfacts_core_parties[partyfacts_core_parties["partyfacts_id"]==3712]

,country,partyfacts_id,technical,name_short,name,name_english,name_other,year_first,year_last,share,share_year,new,wikipedia,description,comment,created,modified
66,AND,3712,NaN,APC,Andorra pel Canvi,Andorra for Change,NaN,2009,2011.0,12.3,2009.0,NaN,https://en.wikipedia.org/wiki/Andorra_for_Change,NaN,NaN,2015-09-08 08:50:18.456268+00:00,2015-11-07 18:32:25.547110+00:00


In [172]:
# polcon.describe()
polcon

,cyear,ccode,cnts_country,polity_country,icrg_country,ctrynm,year,polconiii,polconv,polconvj,...,party11nameupper,p11u,party12nameupper,p12u,party13nameupper,p13u,party14nameupper,p14u,party15nameupper,p15u
0,21800,2,US,United States,NaN,USA,1800.0,0.392401,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
1,21801,2,US,United States,NaN,USA,1801.0,0.385204,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
2,21802,2,US,United States,NaN,USA,1802.0,0.385204,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
3,21803,2,US,United States,NaN,USA,1803.0,0.301785,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
4,21804,2,US,United States,NaN,USA,1804.0,0.301785,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17943,9902012,990,SAMOA,NaN,NaN,WSM,2012.0,0.265319,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
17944,9902013,990,SAMOA,NaN,NaN,WSM,2013.0,0.265319,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
17945,9902014,990,SAMOA,NaN,NaN,WSM,2014.0,0.265319,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
17946,9902015,990,SAMOA,NaN,NaN,WSM,2015.0,0.265319,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0


In [32]:
polcon_ids.set_index("party_id").loc[polcon_to_pf_ids.index]

,ctrynm,party,hog_first,first,last,share_max,share_year,country
1033,AFG,Islamic Society,NaN,1994,1994,23.7,1993,AFG
1046,AFG,People's Democratic Party,NaN,1979,1992,100.0,1988,AFG
1055,ALB,Democratic Party,1993.0,1993,2012,65.7,1992,ALB
1069,ALB,Party of Labour,1945.0,1943,1992,100.0,1959,ALB
1070,ALB,Republican,NaN,1993,2012,7.9,2005,ALB
...,...,...,...,...,...,...,...,...
8525,ZIM,African National Union,1980.0,1980,2012,98.3,1995,ZWE
8527,ZIM,African People's Union,NaN,1980,1990,25.0,1979,ZWE
8529,ZIM,Centre,NaN,1970,1974,10.6,1969,ZWE
8534,ZIM,Movement for Democratic Change,NaN,2001,2012,47.5,2000,ZWE


In [175]:
polcon.groupby("gov1me")["countryname"].apply(lambda x: len(x.unique())).sort_values().tail(50)

polcon.groupby(
    
    "party1namelower"]

KeyError: 'gov1me'

In [176]:
polcon.groupby("party1namelower")["ctrynm"].apply(lambda x: len(x.unique())).sort_values().tail(50)



party1namelower
Revolutionary Party              2
Party of Our Land                2
Whigs                            2
Patriotic Front                  2
Reform                           2
People's Revolutionary Party     2
Social Christian                 2
Democratic Left                  2
National Progressive Front       2
Catholic People's                2
Agrarian                         2
National Democratic Alliance     2
Progressive Conservative         2
New Aspiration                   2
National Democratic              2
Alliance                         3
Monarchist                       3
Progressive                      3
National                         3
Labour Party                     3
Christian Democrats              3
Democrat                         3
Nationalists                     3
Christian Democrat               3
People's Democratic Party        3
Centre                           3
Catholic                         3
Republican                       3
Radi